In [1]:
import numpy as np
import pandas as pd
from typing import List
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

BASE_PATH = "/media/bruno/Arquivos/Desenvolvimento/LaranjaLima"

## Quanto um cliente de um determinado perfil está disposto a gastar?

A ideia é ter uma equação que permita alteração de parâmetros para avaliar o valor que um determinado cliente gastaria em um dado cenário (definido pelos parâmetros). Nesse sentido, nossa target é o campo "Total Pago (R$)".

O cenário é o conjunto de parâmetros que nós previamente escolhemos para avaliar o potencial de gasto. Em outras palavras, as variáveis que utilizaremos para prever a nossa target. Dessa forma, ele será definido pelos campos:
- Perfil do cliente
- Categoria de item em campanha de marketing
- Desconto total oferecido
- Tempo gasto para decisão
- Tempo desde a última compra

In [19]:
class Pipe:
    def __init__(self, df: pd.DataFrame) -> None:
        self.__base_data = df
        self.__encoder = OneHotEncoder(sparse_output=False)
        self.__imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
        self.__scaler = StandardScaler()
        self.__categorical_columns: List[str] = None
        self.__numeric_columns: List[str] = None
        self.__target_column: str = None

    def set_categorical_columns(self, columns: List[str]) -> None:
        self.__categorical_columns = columns

    def set_numeric_columns(self, columns: List[str]) -> None:
        self.__numeric_columns = columns

    def set_target_column(self, column: str) -> None:
        self.__target_column = column

    def get_categorical_columns(self) -> List[str]:
        return [c for c in self.__categorical_columns]
    
    def get_numeric_columns(self) -> List[str]:
        return [c for c in self.__numeric_columns]

    def split(self, test=0.3) -> None:
        self.train, self.test = train_test_split(self.__base_data, test_size=test, random_state=666)

    def fit_preprocess(self) -> None:
        np_cat, np_num = self.train[self.__categorical_columns].values, self.train[self.__numeric_columns].values
        self.__encoder.fit(np_cat)
        self.__imputer.fit(np_num)
        self.__scaler.fit(np_num)

    def transform_data(self, df: pd.DataFrame) -> np.ndarray:
        np_cat, np_num = df[self.__categorical_columns].values, df[self.__numeric_columns].values
        np_cat = self.__encoder.transform(np_cat)
        np_num = self.__imputer.transform(np_num)
        np_num = self.__scaler.transform(np_num)
        return np.concatenate((np_cat, np_num), axis=1)
    
    def train_model(self, model) -> None:
        x_train = self.transform_data(self.train)
        y_train = self.train[self.__target_column].values
        model.fit(x_train, y_train)

        x_test = self.transform_data(self.test)
        y_test = self.test[self.__target_column].values

        print(f"MAE treino: {mean_absolute_error(y_train, model.predict(x_train))}")
        print(f"MAE teste: {mean_absolute_error(y_test, model.predict(x_test))}")

    def predict(self, df: pd.DataFrame, model) -> np.ndarray:
        x = self.transform_data(df)
        return model.predict(x)

In [20]:
dataset = pd.read_csv(f"{BASE_PATH}/data/ml/ecommerce - python - gasto dos clientes.csv")
pipe = Pipe(dataset)
pipe.split(test=0.3)

pipe.set_target_column("Total Pago (R$)")
pipe.set_categorical_columns(["Perfil do Cliente", "Campanha em Curso"])
pipe.set_numeric_columns([
    "Desconto Total (%)",
    "Tempo desde a Última Compra (dias)",
    "Tempo para Finalizar Compra (min)"
])
pipe.fit_preprocess()

print("Testando modelo linear")
linear_model = SGDRegressor(loss="squared_error", penalty="l2", alpha=0.001, eta0=0.001, learning_rate="invscaling")
pipe.train_model(linear_model)

print("\nTestando modelo de máquinas de suporte")
svm_model = SVR(kernel="poly", C=1, epsilon=0.1, degree=5)
pipe.train_model(svm_model)

print("\nTestando modelo de árvore de decisão")
dt_model = DecisionTreeRegressor()
pipe.train_model(dt_model)

Testando modelo linear
MAE treino: 1219.7713183373567
MAE teste: 1260.3891670071464

Testando modelo de máquinas de suporte
MAE treino: 1246.5894557083
MAE teste: 1388.836666042846

Testando modelo de árvore de decisão
MAE treino: 0.0
MAE teste: 1632.0538924731184


In [26]:
columns = pipe.get_categorical_columns()
columns.extend(pipe.get_numeric_columns())
cases = [
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 30],
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 20],
    ["Médio ticket e alta frequência", "Eletrodomésticos", 1, 10, 5],

    ["Médio ticket e alta frequência", "Livros", 1, 10, 30],
    ["Médio ticket e alta frequência", "Livros", 1, 10, 20],
    ["Médio ticket e alta frequência", "Livros", 1, 10, 5],

    ["Médio ticket e alta frequência", "Livros", 1, 10, 30],
    ["Médio ticket e alta frequência", "Livros", 1, 20, 30],
    ["Médio ticket e alta frequência", "Livros", 1, 30, 30]
]
pipe.predict(pd.DataFrame(cases, columns=columns), linear_model)

array([1661.26079669, 1739.04183209, 1855.71338519, 1113.12703583,
       1190.90807123, 1307.57962433, 1113.12703583, 1115.08478219,
       1117.04252856])